In [1]:
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing training dataset
df = pd.read_csv('file:///C:/Users/suhas/OneDrive/Desktop/digitalmarketingandanalytics/Final Project/Namrata Rathore/train.csv')
df = df.sample(frac=0.4)

In [3]:

#using songs and members dataset

songs = pd.read_csv('file:///C:/Users/suhas/OneDrive/Desktop/digitalmarketingandanalytics/Final Project/Namrata Rathore/songs.csv')
df = pd.merge(df, songs, on='song_id', how='left')
del songs

# Load and join songs data
members = pd.read_csv('file:///C:/Users/suhas/OneDrive/Desktop/digitalmarketingandanalytics/Final Project/Namrata Rathore/members.csv')
df = pd.merge(df, members, on='msno', how='left')
del members

In [4]:

df.isnull().sum()/df.isnull().count()*100

msno                       0.000000
song_id                    0.000000
source_system_tab          0.335687
source_screen_name         5.612770
source_type                0.292209
target                     0.000000
song_length                0.001152
genre_ids                  1.600187
artist_name                0.001152
composer                  22.740546
lyricist                  43.115257
language                   0.001728
city                       0.000000
bd                         0.000000
gender                    40.130777
registered_via             0.000000
registration_init_time     0.000000
expiration_date            0.000000
dtype: float64

In [5]:
#removing NA values from df

for i in df.select_dtypes(include=['object']).columns:
    df[i][df[i].isnull()] = 'unknown'
df = df.fillna(value=0)

In [6]:

# creating a column of registration_init_date  using month, year and day

df.registration_init_time = pd.to_datetime(df.registration_init_time, format='%Y%m%d', errors='ignore')
df['registration_init_time_year'] = df['registration_init_time'].dt.year
df['registration_init_time_month'] = df['registration_init_time'].dt.month
df['registration_init_time_day'] = df['registration_init_time'].dt.day

# creating expiration date using year, date, month

df.expiration_date = pd.to_datetime(df.expiration_date,  format='%Y%m%d', errors='ignore')
df['expiration_date_year'] = df['expiration_date'].dt.year
df['expiration_date_month'] = df['expiration_date'].dt.month
df['expiration_date_day'] = df['expiration_date'].dt.day

In [7]:

# conversion into categorical values

df['registration_init_time'] = df['registration_init_time'].astype('category')
df['expiration_date'] = df['expiration_date'].astype('category')

In [8]:

# converting into categorcial values
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')
    
# encoding of categorical values
for col in df.select_dtypes(include=['category']).columns:
    df[col] = df[col].cat.codes

In [9]:
#implementing PCA

import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(df[df.columns[df.columns != 'target']])

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [14]:
# implementing Gaussian model

model = GaussianNB()
model.fit(df[df.columns[df.columns != 'target']], df.target)

GaussianNB(priors=None, var_smoothing=1e-09)

In [15]:
X_array=np.zeros(shape=(len(df)))
len(X_array)

2950967

In [18]:
#K fold cross validation

from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
# 3 is the best split for us
df_val=[]
df_val_target=[]
model = GaussianNB()
for train_indices,val_indices in kf.split(df) :
    model.fit(df.drop(['target'],axis=1).loc[train_indices,:],df.loc[train_indices,'target'])
    X_array=X_array.append(df.loc[val_indices,'target']) 
    #X_array=X_array.vstack(df.loc[val_indices,'target']) 

AttributeError: 'numpy.ndarray' object has no attribute 'vstack'